Weekly and EU data 
Add total for average, compute total and then the average
Change the country's acronym for name
A year has week 53 only if it ends on a Thursday or is a leap year and ends on a Wednesday.
For example:
2019 had a week 53 because it ended on a Tuesday.
2020 and 2021 did not have week 53 because they ended on Thursday and Friday, respectively.
 - drop[ week 53
 - fill in missing weeks with interpolated values if you require all weeks to have values.]

31/12/2018 week of 1 day

In [18]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [19]:
df = pd.read_csv("src/data/analyzed/daily_demand_clean.csv")

#dropping UK for now
df = df[df['country'] != 'UK']
df['demand'] = df['demand'] / 1000000000   #convert to TWh
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d', errors='coerce')

df['week'] = df['date'].dt.isocalendar().week #week 
df['year'] = df['date'].dt.isocalendar().year  # Use ISO year to avoid mismatches at year boundaries e.g. year(31/12/2018) = 2019


In [20]:
# Group by 'week' and all other relevant variables, then count the number of observations per group
group_columns = ['week', 'year', 'country', 'type']  # Adjust these based on your DataFrame's structure
df['count'] = df.groupby(group_columns)['demand'].transform('count')



# Filter rows where the count is 7 or more
df = df[df['count'] == 7]

# Drop the helper 'count' column if no longer needed
df = df.drop(columns=['count'])

In [21]:
#sum total; industry etc to get a European value
df_europe = df.groupby(['type', 'year', 'week', 'date' ])['demand'].sum().reset_index()
df_europe['country'] = 'Europe'  # Assign country as "Europe"



In [22]:
df = pd.concat([df, df_europe], ignore_index=True)

In [23]:
print(df['year'].dtype)
print(df['year'].isna().sum())


UInt32
0


In [24]:
df_weekly = df.groupby(['country', 'type', 'year', 'week'])['demand'].sum().reset_index()
df_weekly = df_weekly[df_weekly['year'] >= 2019]


In [25]:
# Define the years to average
years_to_average = [2019, 2020, 2021]

# Filter for rows corresponding to the specified years
filtered_df = df_weekly[df_weekly['year'].isin(years_to_average)]

# Group by type, week, and country, then calculate the average demand
average_df = (
    filtered_df
    .groupby(['type', 'week','country'], as_index=False)
    .agg({'demand': 'mean'})
)

# Add a new column for the year and set it to "AVG-2019-2021"
average_df['year'] = "AVG-2019-2021"

# Subset 2: Keep rows that are not part of the specified years
remaining_years_df = df_weekly[~df_weekly['year'].isin(years_to_average)]

# Fianl subset: Combine the subsets 
df_with_average = pd.concat([remaining_years_df, average_df], ignore_index=True)
# Replace all 0s in the 'demand' column with NaN
df_with_average['demand'] = df_with_average['demand'].replace(0, np.nan)

In [26]:
# Define a mapping of EU country codes to their full names
country_mapping = {
    "AT": "Austria", "BE": "Belgium", "BG": "Bulgaria", "HR": "Croatia", 
    "CY": "Cyprus", "CZ": "Czech Republic", "DK": "Denmark", "EE": "Estonia", 
    "FI": "Finland", "FR": "France", "DE": "Germany", "GR": "Greece", 
    "HU": "Hungary", "IS": "Iceland", "IE": "Ireland", "IT": "Italy", 
    "LV": "Latvia", "LI": "Liechtenstein", "LT": "Lithuania", "LU": "Luxembourg", 
    "MT": "Malta", "NL": "Netherlands", "NO": "Norway", "PL": "Poland", 
    "PT": "Portugal", "RO": "Romania", "SK": "Slovakia", "SI": "Slovenia", 
    "ES": "Spain", "SE": "Sweden", "CH": "Switzerland", "TR": "Turkey", 
    "UK": "United Kingdom", "Europe": "xEU"
}

# Replace country codes with full names
df_with_average["country"] = df_with_average["country"].map(country_mapping)

# Create the 'countryType' column with full country names
df_with_average["countryType"] = df_with_average["country"] + " - " + df_with_average["type"]


KeyError: 'country_full'

In [ ]:
# Save to a JSON file

file_path = "highcharts/data/weekly_demand_clean.json"
df_with_average.to_json(file_path, orient='records', indent=4)
print(f"The file has been saved as: {file_path}")


The file has been saved as: src/data/analyzed/weekly_demand_clean.json


In [17]:
for (country, type_), group in df_with_average.groupby(['country', 'type']):
    print(f"Country: {country}, Type: {type_}")
    
    group_pivot = group.pivot_table(index='week', columns='year', values='demand', aggfunc='sum')
    
    try:
        if "AVG-2019-2021" in group_pivot.columns:
            plt.plot(group_pivot.index, group_pivot["AVG-2019-2021"], 
                     label='AVG-2019-2021', color='grey', linestyle='-', linewidth=1.8, alpha=0.8)

        if 2022 in group_pivot.columns:
            plt.plot(group_pivot.index, group_pivot[2022], 
                     label='2022', color='grey', linestyle='-', linewidth=1.8, alpha=0.8)

        if 2023 in group_pivot.columns:
            plt.plot(group_pivot.index, group_pivot[2023], 
                     label='2023', color='#D5727D', linestyle='-', linewidth=2)

        if 2024 in group_pivot.columns:
            plt.plot(group_pivot.index, group_pivot[2024], 
                     label='2024', color='#C02C44', linestyle='-', linewidth=2.5)

        group_pivot_2025 = group_pivot.loc[group_pivot.index <= 2]
        if 2025 in group_pivot_2025.columns:
            plt.plot(group_pivot_2025.index, group_pivot_2025[2025], 
                     label='2025', color='#A21636', linestyle='-', linewidth=3)

        # Customize legend and plot title
        plt.legend(title="Year", loc='best', fontsize=10, title_fontsize=12, frameon=False)
        plt.title(f'{country}-{type_} weekly natural gas demand (TWh)')
        plt.ylim(0)
        
        # Save the figure
        plt.savefig(f'src/figures/weekly/{country}_{type_}.png', bbox_inches='tight', dpi=300)
        plt.close()
    
    except Exception as e:
        print(f"Error processing {country}-{type_}: {e}")


Country: AT, Type: power
Country: AT, Type: total
Country: BE, Type: household
Country: BE, Type: industry
Country: BE, Type: power
Country: BE, Type: total
Country: BG, Type: power
Country: BG, Type: total
Country: CZ, Type: power
Country: DE, Type: power
Country: DE, Type: total
Country: DK, Type: power
Country: DK, Type: total
Country: EE, Type: power
Country: EE, Type: total
Country: ES, Type: power
Country: ES, Type: total
Country: Europe, Type: household
Country: Europe, Type: industry
Country: Europe, Type: industry-power
Country: Europe, Type: power
Country: Europe, Type: total
Country: FI, Type: power
Country: FR, Type: household
Country: FR, Type: industry
Country: FR, Type: power
Country: FR, Type: total
Country: GR, Type: power
Country: HR, Type: power
Country: HR, Type: total
Country: HU, Type: household
Country: HU, Type: industry
Country: HU, Type: industry-power
Country: HU, Type: power
Country: HU, Type: total
Country: IT, Type: household
Country: IT, Type: industry
Co